In [1]:
import pandas as pd
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import K2Score

In [2]:
characteristics_df = pd.read_csv('data/characteristics_prepped.csv', encoding='latin1')
places_df = pd.read_csv('data/places_prepped.csv', encoding='latin1')
users_df = pd.read_csv('data/users_prepped.csv', encoding='latin1')
vehicles_df = pd.read_csv('data/vehicles_prepped.csv', encoding='latin1')

## Vehicles + users merge

Vechicles and users table might contain multiple rows for the same Num_Acc id. (Because there might be multiple people or vehicles involved in the accident)

In [3]:
vehicles_df.shape

(1433389, 8)

In [4]:
users_df.shape

(1876005, 13)

In [5]:
vehicles_users_df = pd.merge(vehicles_df, users_df, on='Num_Acc', how='outer', suffixes=('_vehicles', '_users'))

Filter rows that that contains a user-vehicle pair that does not match.

In [6]:
vehicles_users_df = vehicles_users_df[vehicles_users_df['num_veh_vehicles'] == vehicles_users_df['num_veh_users']]

In [7]:
vehicles_users_df.head()

,Num_Acc,Occupants,Mobile Obstacle,Shock,num_veh_vehicles,Vehicle_Category,Fix_obstacle_type,Maneuver_category,Seat,User type,Severity,Sex,Trip,Pedestrian location,Pedestrian action,Pedestrian alone,Birth year,num_veh_users,Safety equipment existence,Safety equipment use
0,200500000001,1,2.0,1.0,A01,8,0,1,1,1,2,1,1,0,0,0,1976,A01,1,1
7,200500000001,1,2.0,8.0,B02,8,0,1,1,1,3,2,3,0,0,0,1968,B02,1,1
8,200500000001,1,2.0,8.0,B02,8,0,1,2,2,1,1,0,0,0,0,1964,B02,1,1
9,200500000001,1,2.0,8.0,B02,8,0,1,4,2,1,1,0,0,0,0,2004,B02,3,1
10,200500000001,1,2.0,8.0,B02,8,0,1,5,2,1,1,0,0,0,0,1998,B02,1,1


In [8]:
vehicles_users_df['num_veh_vehicles'].isna().sum()

0

In [9]:
vehicles_users_df['num_veh_users'].isna().sum()

0

In [10]:
vehicles_users_df.drop(['num_veh_users', 'num_veh_vehicles'], axis=1, inplace=True)

In [11]:
vehicles_users_df.shape

(1875983, 18)

There are no vehicles without users and no users without vehicles.

## Characteristics + places merge

In [12]:
places_df.shape

(839985, 11)

In [13]:
characteristics_df.shape

(839985, 11)

In [14]:
characteristics_places_df = pd.merge(places_df, characteristics_df, on='Num_Acc', how='inner')

In [15]:
characteristics_places_df.shape

(839985, 21)

## Merge all

In [16]:
merged_df = pd.merge(characteristics_places_df, vehicles_users_df, on='Num_Acc', how='inner')

In [17]:
merged_df.shape

(1875983, 38)

In [19]:
merged_df.head()

,Num_Acc,road_type,traffic_type,num_lanes,bike_lane_type,terrain_type,curve_type,condition,infrastructure,situ,...,User type,Severity,Sex,Trip,Pedestrian location,Pedestrian action,Pedestrian alone,Birth year,Safety equipment existence,Safety equipment use
0,201600000001,3.0,2.0,0.0,0.0,1.0,3.0,1.0,0.0,1.0,...,1,1,2,0,0,0,0,1983,1,1
1,201600000001,3.0,2.0,0.0,0.0,1.0,3.0,1.0,0.0,1.0,...,1,3,1,9,0,0,0,2001,2,1
2,201600000002,3.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,...,1,3,1,5,0,0,0,1960,1,1
3,201600000002,3.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,...,2,3,1,0,0,0,0,2000,1,1
4,201600000002,3.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,...,2,3,2,0,0,0,0,1962,1,1


In [18]:
scoring_method = K2Score(data=merged_df)
est = HillClimbSearch(data=merged_df)
estimated_model = est.estimate(
    scoring_method=scoring_method, max_indegree=4, max_iter=int(1e4)
)

  0%|          | 0/10000 [00:00<?, ?it/s]

MemoryError: Unable to allocate 5.78 GiB for an array with shape (924, 839985) and data type float64